In [1]:
from pybaseball import statcast
import pandas as pd

In [3]:
data_list = []
for year in ['2010', '2011', '2012', '2013', '2014', '2015']:
    
    data = pd.read_csv(f"/home/chris/sports/baseball/data/pitch_by_pitch_{year}.csv")
    
    cols_to_keep = ['game_pk', 'inning', 'inning_topbot', 'at_bat_number', 'outs_when_up', 
                    'on_1b', 'on_2b', 'on_3b', 'bat_score', 'events']

    for col in ['game_pk', 'inning', 'at_bat_number', 'outs_when_up', 'bat_score']:
        data[col] = data[col].astype(int)

    for col in ['on_1b', 'on_2b', 'on_3b']:
        data[col] = data[col] == data[col]

    data = data[data['events'] == data['events']]

    data = data[cols_to_keep]

    data = data[data['inning'] <= 8]
    
    data_list.append(data)
    
data = pd.concat(data_list)

print(data.shape)
data.head()

/home/chris/.venvs/sports/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,44,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(991991, 10)


,game_pk,inning,inning_topbot,at_bat_number,outs_when_up,on_1b,on_2b,on_3b,bat_score,events
17,266238,8,Bot,68,2,True,False,False,3,field_out
25,266238,8,Bot,67,1,True,False,False,3,field_out
29,266238,8,Bot,66,0,True,False,False,3,field_out
32,266238,8,Bot,65,0,False,False,False,3,single
35,266238,8,Bot,64,0,False,False,False,2,home_run


In [4]:
game_state_dict = {
    (0, False, False, False): 1,
    (1, False, False, False): 2,
    (2, False, False, False): 3,
    (0, True, False, False): 4,
    (1, True, False, False): 5,
    (2, True, False, False): 6,
    (0, False, True, False): 7,
    (1, False, True, False): 8,
    (2, False, True, False): 9,
    (0, True, True, False): 10,
    (1, True, True, False): 11,
    (2, True, True, False): 12,
    (0, False, False, True): 13,
    (1, False, False, True): 14,
    (2, False, False, True): 15,
    (0, True, False, True): 16,
    (1, True, False, True): 17,
    (2, True, False, True): 18,
    (0, False, True, True): 19,
    (1, False, True, True): 20,
    (2, False, True, True): 21,
    (0, True, True, True): 22,
    (1, True, True, True): 23,
    (2, True, True, True): 24,
}

def get_game_state(row):
    
    return game_state_dict[row['outs_when_up'], row['on_1b'], row['on_2b'], row['on_3b']]

data['game_state'] = data.apply(get_game_state, axis=1)

data = data[['game_pk', 'inning', 'inning_topbot', 'at_bat_number', 'game_state', 'bat_score', 'events']]

data.sort_values(by=['game_pk', 'inning_topbot', 'inning', 'at_bat_number'], inplace=True)

print(data.shape)
data.head()

(991991, 7)


,game_pk,inning,inning_topbot,at_bat_number,game_state,bat_score,events
710030,263809,1,Bot,7,1,0,walk
710025,263809,1,Bot,8,4,0,single
710023,263809,1,Bot,9,16,0,single
710020,263809,1,Bot,10,10,1,single
710017,263809,1,Bot,11,22,1,strikeout


In [5]:
print(len(data['game_pk'].unique().tolist()))

14579


In [ ]:
game_state_runs = {
    1: {'count': 0, 'runs': 0},
    2: {'count': 0, 'runs': 0},
    3: {'count': 0, 'runs': 0},
    4: {'count': 0, 'runs': 0},
    5: {'count': 0, 'runs': 0},
    6: {'count': 0, 'runs': 0},
    7: {'count': 0, 'runs': 0},
    8: {'count': 0, 'runs': 0},
    9: {'count': 0, 'runs': 0},
    10: {'count': 0, 'runs': 0},
    11: {'count': 0, 'runs': 0},
    12: {'count': 0, 'runs': 0},
    13: {'count': 0, 'runs': 0},
    14: {'count': 0, 'runs': 0},
    15: {'count': 0, 'runs': 0},
    16: {'count': 0, 'runs': 0},
    17: {'count': 0, 'runs': 0},
    18: {'count': 0, 'runs': 0},
    19: {'count': 0, 'runs': 0},
    20: {'count': 0, 'runs': 0},
    21: {'count': 0, 'runs': 0},
    22: {'count': 0, 'runs': 0},
    23: {'count': 0, 'runs': 0},
    24: {'count': 0, 'runs': 0},
}

game_pk_list = data['game_pk'].unique().tolist()

for j, game_pk in enumerate(game_pk_list):
    
    if j % 100 == 0:
        print(f"Game {j} of {len(game_pk_list)}")
    for inning_topbot in ['Top', 'Bot']:
        for inning in [1, 2, 3, 4, 5, 6, 7, 8]:

            data_temp = data[(data['game_pk'] == game_pk) & (data['inning'] == inning) & (data['inning_topbot'] == inning_topbot)]

            for i in range(len(data_temp)):
                game_state = data_temp.iloc[i]['game_state']
                final_score = data_temp.iloc[-1]['bat_score']
                state_score = data_temp.iloc[i]['bat_score']
                game_state_runs[game_state]['count'] += 1
                game_state_runs[game_state]['runs'] += (final_score - state_score)
            
game_state_runs

In [ ]:
for i in list(game_state_runs.keys()):
    print(i, game_state_runs[i]['runs'] / game_state_runs[i]['count'])